In [ ]:
import os,re,h5py
import requests
import zipfile

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import scipy, sys, os, pyproj, glob
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import pointCollection as pc
import math

# Import some of the scripts that we have written
import sys
sys.path.append("/home/jovyan/surface_velocity/scripts")
from loading_scripts import atl06_to_dict


# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
#From Ben Smith's code loading in .tif file, running into issues likely with directories
data_root='/srv/shared/surface_velocity/FIS_Velocity/'
#spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-65, -86, -55, -81])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
#Originally tried to load data from a local directory, should change to shared directory
Measures_vx=pc.grid.data().from_geotif(os.path.join(data_root,'Measures2_FIS_Vx.tif'), bounds=[XR, YR])
Measures_vy=pc.grid.data().from_geotif(os.path.join(data_root,'Measures2_FIS_Vy.tif'), bounds=[XR, YR])

In [ ]:

def add_surface_velocity_to_is2_dict(is2_dict, ):
    
    vx = Measures_vx.interp(is2_dict['x'],is2_dict['y'])
    vy = Measures_vy.interp(is2_dict['x'],is2_dict['y'])

    #Solve for angle to rotate Vy to be along track and Vx to be across track
    import math
    xL=abs((is2_dict['x'][0])-(is2_dict['x'][1]))
    yL=abs((is2_dict['y'][0])-(is2_dict['y'][1]))

    #decides if is descending or ascending path
    if is2_dict['x'][0]-is2_dict['x'][1] < 0:

        theta_rad=math.atan(xL/yL)
        #theta_deg=theta_rad*180/math.pi
        is2_dict['v_along']=vy/math.cos(theta_rad)
        is2_dict['v_across']=vx/math.cos(theta_rad)

    else:
    
        theta_rad=math.atan(xL/yL)
        #theta_deg=theta_rad*180/math.pi
        is2_dict['v_along']=vy/math.sin(theta_rad)
        is2_dict['v_across']=vx/math.sin(theta_rad)
        
    return is2_dict 

In [ ]:
data_root='/srv/shared/surface_velocity/'
field_dict={None:['delta_time','latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}

filename = glob.glob(os.path.join(data_root, 'FIS_ATL06', f'*ATL06_*_*_003*.h5'))


beams = ['/gt1r','/gt1l','/gt2r','/gt2l','/gt3r','/gt3l']
D=[]
for file in filename:
    B = ATL06_to_dict(file, dataset_dict=field_dict)
        
    add_surface_velocity_to_is2_dict(B)
    D.append(B)
        